In [3]:
# Install required packages
!pip install openai
!pip install python-dotenv

import os
import openai
import time
from datetime import datetime

# Azure OpenAI Configuration
openai.api_type = "azure"
openai.api_base = "Type Azure OpenAI Endpoint"
openai.api_key = "Type Azure OpenAI Key"
openai.api_version = "2024-05-01-preview"
deployment_name = "gpt-35-turbo-instruct"

def get_completion(prompt, retry_delay=10):
    """
    Get completion with retry logic and rate limiting
    
    Args:
        prompt (str): The prompt to send to the API
        retry_delay (int): Delay in seconds before retrying
    """
    while True:
        try:
            response = openai.Completion.create(
                engine=deployment_name,
                prompt=prompt,
                max_tokens=800,
                temperature=0
            )
            return response.choices[0].text.strip()
        except Exception as e:
            if "rate limit" in str(e).lower():
                print(f"\nRate limit reached. Waiting {retry_delay} seconds...")
                time.sleep(retry_delay)
                continue
            else:
                print(f"Error: {str(e)}")
                return None

# Step 1: Test basic completion
print("Step 1.1: Testing basic completion...")
test_response = get_completion("What is 1+1?")
print(f"Test response: {test_response}")

# Add delay before next request
time.sleep(10)
print("\nWaiting 10 seconds before next request...")

# Step 2: Test email translation
print("\nStep 1.2: Testing email translation...")
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

style = """American English \
in a calm and respectful tone
"""

prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

translation = get_completion(prompt)
print("\nTranslated customer email:")
print(translation)

print("\nStep 1 completed successfully!")

Step 1.1: Testing basic completion...
Test response: 1+1 is equal to 2.

Waiting 10 seconds before next request...

Step 1.2: Testing email translation...

Rate limit reached. Waiting 10 seconds...

Rate limit reached. Waiting 10 seconds...

Rate limit reached. Waiting 10 seconds...

Rate limit reached. Waiting 10 seconds...

Rate limit reached. Waiting 10 seconds...

Translated customer email:
I am quite upset that my blender lid flew off and made a mess of my kitchen walls with smoothie! To add to my frustration, the warranty does not cover the cost of cleaning up my kitchen. I would greatly appreciate your assistance at this moment, my friend.

Step 1 completed successfully!


In [4]:
# Install required packages
!pip install langchain
!pip install langchain-community

import time
from langchain_community.llms import AzureOpenAI
from langchain.prompts import PromptTemplate

# Initialize LangChain model (using existing Azure OpenAI configuration)
llm = AzureOpenAI(
    deployment_name=deployment_name,
    model_name="gpt-35-turbo-instruct",
    openai_api_base=openai.api_base,
    openai_api_key=openai.api_key,
    openai_api_version="2024-05-01-preview",
    openai_api_type="azure"
)

print("Step 2.1: Setting up LangChain template...")
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template = PromptTemplate(
    input_variables=["style", "text"],
    template=template_string
)

# Test first translation
print("\nStep 2.2: Testing customer email translation...")
customer_style = "American English in a calm and respectful tone"
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

formatted_prompt = prompt_template.format(
    style=customer_style,
    text=customer_email
)

print("Waiting 10 seconds before API call...")
time.sleep(10)
response = llm.predict(formatted_prompt)
print("\nTranslated response:")
print(response)

# Test second translation with delay
print("\nStep 2.3: Testing pirate style translation...")
print("Waiting 10 seconds before next API call...")
time.sleep(10)

service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = "a polite tone that speaks in English Pirate"

formatted_prompt = prompt_template.format(
    style=service_style_pirate,
    text=service_reply
)

response = llm.predict(formatted_prompt)
print("\nPirate style response:")
print(response)

print("\nStep 2 completed successfully!")

/tmp/ipykernel_12005/2856364615.py:10: LangChainDeprecationWarning: The class `AzureOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAI`.
  llm = AzureOpenAI(


Step 2.1: Setting up LangChain template...

Step 2.2: Testing customer email translation...
Waiting 10 seconds before API call...


/tmp/ipykernel_12005/2856364615.py:50: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm.predict(formatted_prompt)



Translated response:

I am quite upset that my blender lid flew off and left smoothie all over my kitchen walls! To make matters worse, the warranty does not cover the cost of cleaning up my kitchen. I could really use your assistance at this moment, my friend.

Step 2.3: Testing pirate style translation...
Waiting 10 seconds before next API call...

Pirate style response:

Ahoy there matey! I be sorry to inform ye that the warranty does not be coverin' the cost o' cleanin' yer galley, as 'tis yer own fault fer misusin' yer blender. Ye be forgettin' to put the lid on before startin' the blender, savvy? Tough luck! Fare thee well!

Step 2 completed successfully!


In [5]:
# Import required modules for Output Parser
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
import time
import json

print("Step 3.1: Setting up Output Parser...")
# Define schemas
response_schemas = [
    ResponseSchema(
        name="gift",
        description="Was the item purchased as a gift for someone else? \
        Answer true or false (lowercase)."
    ),
    ResponseSchema(
        name="delivery_days",
        description="How many days did it take for the product to arrive? \
        Answer with a number. If not found, answer -1."
    ),
    ResponseSchema(
        name="price_value",
        description="Extract any sentences about the value or price."
    )
]

# Create parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

# Create review template
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer true or false (lowercase).

delivery_days: How many days did it take for the product \
to arrive? Answer with a number. If not found, answer -1.

price_value: Extract any sentences about the value or price.

text: {text}

{format_instructions}

IMPORTANT: Ensure the response is valid JSON with proper formatting.
"""

# Create prompt template
prompt = PromptTemplate(
    template=review_template,
    input_variables=["text"],
    partial_variables={"format_instructions": format_instructions}
)

print("\nStep 3.2: Testing Output Parser...")
# Test with a review
test_review = """
This leaf blower is pretty amazing. It has four settings:
candle blower, gentle breeze, windy city, and tornado. 
It arrived in two days, just in time for my wife's 
anniversary present. I think my wife liked it so much she was speechless. 
It's slightly more expensive than the other leaf blowers 
out there, but I think it's worth it for the extra features.
"""

print("Waiting 10 seconds before API call...")
time.sleep(10)

formatted_prompt = prompt.format(text=test_review)
response = llm.predict(formatted_prompt)
print("\nRaw response:")
print(response)

try:
    parsed_output = output_parser.parse(response)
    print("\nParsed output:")
    print("Gift:", parsed_output['gift'])
    print("Delivery Days:", parsed_output['delivery_days'])
    print("Price Value:", parsed_output['price_value'])
except Exception as e:
    print(f"\nError parsing response: {str(e)}")

print("\nStep 3 completed successfully!")

Step 3.1: Setting up Output Parser...

Step 3.2: Testing Output Parser...
Waiting 10 seconds before API call...

Raw response:

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```

Parsed output:
Gift: True
Delivery Days: 2
Price Value: It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.

Step 3 completed successfully!
